In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [43]:
data = pd.read_csv("cleaned_data.csv") 

In [44]:
data.head()

,Unnamed: 0,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,...,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Cloud9am,Cloud3pm,AvgTemp,AvgPressure,RainToday,RainTomorrow
0,0,2008,2,13.4,22.9,0.6,6.484195,7.001431,13,44.0,...,20.0,24.0,71.0,22.0,8.000000,5.121202,19.35,1007.40,0.0,0.0
1,1,2008,2,7.4,25.1,0.0,6.278940,11.029074,14,44.0,...,4.0,22.0,44.0,25.0,1.853901,2.553898,20.75,1009.20,0.0,0.0
2,2,2008,2,12.9,25.7,0.0,8.681495,11.784566,15,46.0,...,19.0,26.0,38.0,30.0,2.016364,2.000000,22.10,1008.15,0.0,0.0
3,3,2008,2,9.2,28.0,0.0,6.777574,11.395051,4,24.0,...,11.0,9.0,45.0,16.0,1.236367,2.068572,22.30,1015.20,0.0,0.0
4,4,2008,2,17.5,32.3,1.0,7.437996,5.751086,13,41.0,...,7.0,20.0,82.0,33.0,7.000000,8.000000,23.75,1008.40,0.0,0.0


In [45]:
data.isnull().sum()

Unnamed: 0       0
Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Cloud9am         0
Cloud3pm         0
AvgTemp          0
AvgPressure      0
RainToday        0
RainTomorrow     0
dtype: int64

In [46]:
data = data.drop(data.columns[0], axis=1)

In [47]:
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Cloud9am,Cloud3pm,AvgTemp,AvgPressure,RainToday,RainTomorrow
0,2008,2,13.4,22.9,0.6,6.484195,7.001431,13,44.0,13,...,20.0,24.0,71.0,22.0,8.000000,5.121202,19.35,1007.40,0.0,0.0
1,2008,2,7.4,25.1,0.0,6.278940,11.029074,14,44.0,6,...,4.0,22.0,44.0,25.0,1.853901,2.553898,20.75,1009.20,0.0,0.0
2,2008,2,12.9,25.7,0.0,8.681495,11.784566,15,46.0,13,...,19.0,26.0,38.0,30.0,2.016364,2.000000,22.10,1008.15,0.0,0.0
3,2008,2,9.2,28.0,0.0,6.777574,11.395051,4,24.0,9,...,11.0,9.0,45.0,16.0,1.236367,2.068572,22.30,1015.20,0.0,0.0
4,2008,2,17.5,32.3,1.0,7.437996,5.751086,13,41.0,1,...,7.0,20.0,82.0,33.0,7.000000,8.000000,23.75,1008.40,0.0,0.0


In [48]:
X = data.drop(columns=['RainTomorrow']).values  
y = data['RainTomorrow'].values.reshape(-1, 1)  

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [50]:
# Define sigmoid activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Initialize weights and biases
np.random.seed(42)
input_dim = X_train_scaled.shape[1]
hidden_dim = 64
output_dim = 1
W1 = np.random.randn(input_dim, hidden_dim)
b1 = np.zeros((1, hidden_dim))
W2 = np.random.randn(hidden_dim, output_dim)
b2 = np.zeros((1, output_dim))

# Training parameters
learning_rate = 0.01
epochs = 20
batch_size = 32
n_batches = len(X_train_scaled) // batch_size

# Training the model
for epoch in range(epochs):
    for i in range(n_batches):
        # Mini-batch
        X_batch = X_train_scaled[i * batch_size:(i + 1) * batch_size]
        y_batch = y_train[i * batch_size:(i + 1) * batch_size]

        # Forward propagation
        z1 = np.dot(X_batch, W1) + b1
        a1 = np.maximum(0, z1)  # ReLU activation
        z2 = np.dot(a1, W2) + b2
        y_pred = sigmoid(z2)

        # Compute loss
        loss = -np.mean(y_batch * np.log(y_pred) + (1 - y_batch) * np.log(1 - y_pred))

        # Backpropagation
        dz2 = y_pred - y_batch
        dW2 = np.dot(a1.T, dz2)
        db2 = np.sum(dz2, axis=0, keepdims=True)
        da1 = np.dot(dz2, W2.T)
        dz1 = da1 * (z1 > 0)  # ReLU derivative
        dW1 = np.dot(X_batch.T, dz1)
        db1 = np.sum(dz1, axis=0, keepdims=True)

        # Update weights and biases
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2
        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1

# Evaluate the model
z1 = np.dot(X_test_scaled, W1) + b1
a1 = np.maximum(0, z1)  # ReLU activation
z2 = np.dot(a1, W2) + b2
y_pred = sigmoid(z2)
y_pred_class = (y_pred > 0.5).astype(int)
accuracy = np.mean(y_pred_class == y_test)

print("Test Accuracy:", accuracy)

C:\Users\prath\AppData\Local\Temp\ipykernel_23120\3094883321.py:35: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(y_batch * np.log(y_pred) + (1 - y_batch) * np.log(1 - y_pred))
C:\Users\prath\AppData\Local\Temp\ipykernel_23120\3094883321.py:35: RuntimeWarning: invalid value encountered in multiply
  loss = -np.mean(y_batch * np.log(y_pred) + (1 - y_batch) * np.log(1 - y_pred))


Test Accuracy: 0.8955726660250241


In [51]:
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Cloud9am,Cloud3pm,AvgTemp,AvgPressure,RainToday,RainTomorrow
0,2008,2,13.4,22.9,0.6,6.484195,7.001431,13,44.0,13,...,20.0,24.0,71.0,22.0,8.000000,5.121202,19.35,1007.40,0.0,0.0
1,2008,2,7.4,25.1,0.0,6.278940,11.029074,14,44.0,6,...,4.0,22.0,44.0,25.0,1.853901,2.553898,20.75,1009.20,0.0,0.0
2,2008,2,12.9,25.7,0.0,8.681495,11.784566,15,46.0,13,...,19.0,26.0,38.0,30.0,2.016364,2.000000,22.10,1008.15,0.0,0.0
3,2008,2,9.2,28.0,0.0,6.777574,11.395051,4,24.0,9,...,11.0,9.0,45.0,16.0,1.236367,2.068572,22.30,1015.20,0.0,0.0
4,2008,2,17.5,32.3,1.0,7.437996,5.751086,13,41.0,1,...,7.0,20.0,82.0,33.0,7.000000,8.000000,23.75,1008.40,0.0,0.0


In [61]:
import numpy as np
from sklearn.preprocessing import StandardScaler

mean_values = data.mean().to_dict()

# Function to preprocess user input
def preprocess_input(user_input):
    # Convert input to numpy array and reshape
    input_array = np.array(user_input).reshape(1, -1)
    # Scale the input using the same scaler used during training
    input_scaled = scaler.transform(input_array)
    return input_scaled

# Function to predict rain tomorrow
def predict_rain_tomorrow(input_data):
    # Perform prediction using the trained model
    z1 = np.dot(input_data, W1) + b1
    a1 = np.maximum(0, z1)  # ReLU activation
    z2 = np.dot(a1, W2) + b2
    y_pred = sigmoid(z2)
    return "Yes" if y_pred[0][0] >= 0.5 else "No"

# Features list
features = ["Date", "Location", "MinTemp", "MaxTemp", "Rainfall", "Evaporation", "Sunshine", 
            "WindGustDir", "WindGustSpeed", "WindDir9am", "WindDir3pm", "WindSpeed9am", 
            "WindSpeed3pm", "Humidity9am", "Humidity3pm",  "Cloud9am", 
            "Cloud3pm", "AvgTemp", "AvgPressure", "RainToday"]



# Get user input for each feature
user_input = []
for feature in features:
    default_value = mean_values.get(feature, 0)  # Get the mean value for the feature
    value = input(f"Enter {feature} [default value: {default_value}]: ") or default_value  # Use the mean value if no input provided
    user_input.append(float(value))


# Preprocess user input
input_scaled = preprocess_input(user_input)

# Predict rain tomorrow
prediction = predict_rain_tomorrow(input_scaled)

# Display prediction
print("Rain tomorrow?", prediction)


Enter Date [default value: 2012.7697511343324]:  2008
Enter Location [default value: 23.793523992850268]:  2
Enter MinTemp [default value: 12.194188898284361]:  9.7
Enter MaxTemp [default value: 23.22145157698355]:  31.9
Enter Rainfall [default value: 2.3205416314439757]:  0
Enter Evaporation [default value: 5.095513796408239]:  9.02
Enter Sunshine [default value: 7.433711666275929]:  10.38
Enter WindGustDir [default value: 8.306565378798295]:  6
Enter WindGustSpeed [default value: 39.80336061841595]:  80
Enter WindDir9am [default value: 7.943826481506943]:  9
Enter WindDir3pm [default value: 8.002316788120446]:  7
Enter WindSpeed9am [default value: 14.040561207766899]:  7
Enter WindSpeed3pm [default value: 18.65513640536835]:  28
Enter Humidity9am [default value: 68.93314006871466]:  42
Enter Humidity3pm [default value: 51.599939357215376]:  9
Enter Cloud9am [default value: 4.382486714047216]:  1.99
Enter Cloud3pm [default value: 4.481997239311064]:  3.12
Enter AvgTemp [default value:

Rain tomorrow? Yes
